In [1]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool,ColumnDataSource

# 在 Notebook 中输出
output_notebook()

Loading BokehJS ...

In [2]:
from models import StockDayTrade
from sqlmodel import select, Session
from database import get_engine

In [3]:
results = []
with Session(get_engine()) as session:
    statement = (
            select(StockDayTrade)
            .where(StockDayTrade.ts_code == "603298.SH")
            .order_by(StockDayTrade.id)  # 按 trade_date 降序排序
            .limit(200)  
        )
    results = session.exec(statement).all()
    

In [4]:

ids = range(len(results))
prices = (res.close for res in results)
dates = (res.trade_date for res in results)

source = ColumnDataSource(data={
    "id": list(ids),
    "date": list(dates)[::-1],
    "close": list(prices)[::-1],
})
# 创建图表
p = figure(
    title="Stock Prices Over Time",
    x_axis_label="ID",
    y_axis_label="Close Price",
    tooltips=None,  # 初始不启用全局悬停
)

# 添加折线图
line = p.line("id", "close", source=source, line_width=2, legend_label="Close Price", name="line")

# 添加 Hover 工具
hover = HoverTool(
    renderers=[line],  # 绑定到特定渲染器（line）
    tooltips=[
        ("Date", "@date"),  # 绑定日期
        ("Close Price", "@close"),  # 绑定收盘价
        ("ID", "@id"),  # 可选：显示 ID
    ],
    mode="vline",  # 悬停模式（垂直线）
)

p.add_tools(hover)  # 添加 Hover 工具

# 显示图表
show(p)

In [10]:
import numpy as np

prices = [res.close for res in results]
def create_features(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])  # 取窗口大小的数据作为特征
        y.append(data[i+window_size])   # 下一个点作为目标
    return np.array(X), np.array(y)


# 定义窗口大小
window_size = 3
X, y = create_features(prices[::-1], window_size)

print("Features (X):\n", X)
print("Targets (y):\n", y)

Features (X):
 [[27.5  27.51 26.5 ]
 [27.51 26.5  26.75]
 [26.5  26.75 26.07]
 [26.75 26.07 26.46]
 [26.07 26.46 27.31]
 [26.46 27.31 26.94]
 [27.31 26.94 26.56]
 [26.94 26.56 26.6 ]
 [26.56 26.6  27.42]
 [26.6  27.42 28.47]
 [27.42 28.47 28.71]
 [28.47 28.71 28.8 ]
 [28.71 28.8  28.84]
 [28.8  28.84 28.64]
 [28.84 28.64 28.87]
 [28.64 28.87 30.24]
 [28.87 30.24 30.38]
 [30.24 30.38 31.96]
 [30.38 31.96 31.53]
 [31.96 31.53 32.56]
 [31.53 32.56 31.79]
 [32.56 31.79 31.87]
 [31.79 31.87 30.91]
 [31.87 30.91 29.61]
 [30.91 29.61 30.31]
 [29.61 30.31 29.5 ]
 [30.31 29.5  29.39]
 [29.5  29.39 29.47]
 [29.39 29.47 29.96]
 [29.47 29.96 31.39]
 [29.96 31.39 32.  ]
 [31.39 32.   31.76]
 [32.   31.76 33.4 ]
 [31.76 33.4  33.67]
 [33.4  33.67 33.95]
 [33.67 33.95 33.77]
 [33.95 33.77 33.87]
 [33.77 33.87 33.98]
 [33.87 33.98 32.1 ]
 [33.98 32.1  30.68]
 [32.1  30.68 30.12]
 [30.68 30.12 30.  ]
 [30.12 30.   30.65]
 [30.   30.65 31.3 ]
 [30.65 31.3  30.41]
 [31.3  30.41 29.4 ]
 [30.41 29.4  28.85

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.linear_model import LinearRegression

# 初始化模型
model = LinearRegression()

# 训练模型
model.fit(X_train, y_train)

# 评估模型
score = model.score(X_test, y_test)
print(f"Model R^2 Score: {score}")

Model R^2 Score: 0.9846176725868642


In [15]:
# 使用最新窗口预测下一个点
last_window = prices[::-1][-window_size:]  # 取最后一个窗口
next_point = model.predict([last_window])  # 注意输入需要是 2D

print(f"Next predicted point: {next_point[0]}")

Next predicted point: 19.142806686848463
